# Models creation

Requirements loading

In [ ]:
%run -i ../script/requirements.py
%run -i ../script/requirements_dll.py
%matplotlib inline
pd.set_option('display.max_columns', None)

## Preprocessing

In [ ]:
df = pd.read_csv('../df_completed_order.csv', dtype = {'_CASE_KEY' : str, 'WERKS' : str, 'EBELN' : str, 'EBELP': str, 'ACTIVITY_EN': str, '_SORTING' : int, 'USER_NAME': str})
df['EVENTTIME'] = pd.to_datetime(df['EVENTTIME'])
df['BLDAT'] = pd.to_datetime(df['BLDAT'])
df['BUDAT'] = pd.to_datetime(df['BUDAT'])
df['BPDAT'] = pd.to_datetime(df['BPDAT'])
df['DRDAT'] = pd.to_datetime(df['DRDAT'])
df['LEWED'] = pd.to_datetime(df['LEWED'])
df['SEDAT'] = pd.to_datetime(df['SEDAT'])
df['LCWED'] = pd.to_datetime(df['LCWED'])
df['ITDAT'] = pd.to_datetime(df['ITDAT'])
df = df.rename(columns={'_CASE_KEY' : 'case:concept:name',
                       'ACTIVITY_EN' : 'concept:name',
                       'EVENTTIME' : 'time:timestamp'})
df.loc[df['RKDST'].isnull(), 'RKDST'] = 0
df.loc[df['RKDST'] == 'x', 'RKDST'] = 1
df.loc[df['MAVPR'].isnull(), 'MAVPR'] = 0
df.loc[df['THWMS'].isnull(), 'THWMS'] = 0
df.loc[df['LATEP'].isnull(), 'LATEP'] = 0
df['MAVPR'] = df['MAVPR'].astype(int)
df['THWMS'] = df['THWMS'].astype(int)
df['LATEP'] = df['LATEP'].astype(int)

In [ ]:
df.loc[~df['concept:name'].isin(['Record Invoice','Cleared Invoice', 'Due Date passed', 'Remove Payment Block', 'Set Payment Block']), ['BLDAT','BUDAT','BPDAT']] = pd.NaT
df.loc[df['BLDAT'].isna(), ['BLDAT', 'BUDAT', 'BPDAT']] = df['BLDAT'].iloc[-1]

In [ ]:
# Time
A = [d.date() for d in df['BLDAT']]
B = [d.date() for d in df['BUDAT']]
C = [d.date() for d in df['BPDAT']]
df['time:diff_BUDAT'] = np.busday_count(A, B)
df['time:diff_BPDAT'] = np.busday_count(A, C)
sc_day = StandardScaler()
df['time:diff_BPDAT'] = sc_day.fit_transform(np.array(df['time:diff_BPDAT']).reshape(-1, 1))
df['time:diff_BUDAT'] = sc_day.transform(np.array(df['time:diff_BUDAT']).reshape(-1, 1))

# Time
A = [d.date() for d in df['DRDAT']]
B = [d.date() for d in df['LEWED']]
C = [d.date() for d in df['SEDAT']]
df['time:diff_LEWED'] = np.busday_count(A, B)
df['time:diff_SEDAT'] = np.busday_count(A, C)
sc_day_ship = StandardScaler()
df['time:diff_LEWED'] = sc_day_ship.fit_transform(np.array(df['time:diff_LEWED']).reshape(-1, 1))
df['time:diff_SEDAT'] = sc_day_ship.transform(np.array(df['time:diff_SEDAT']).reshape(-1, 1))

# Time
A = [d.date() for d in df['DRDAT']]
B = [d.date() for d in df['LCWED']]
C = [d.date() for d in df['ITDAT']]
df['time:diff_LCWED'] = np.busday_count(A, B)
df['time:diff_ITDAT'] = np.busday_count(A, C)
sc_day_del = StandardScaler()
df['time:diff_LCWED'] = sc_day_del.fit_transform(np.array(df['time:diff_LCWED']).reshape(-1, 1))
df['time:diff_ITDAT'] = sc_day_del.transform(np.array(df['time:diff_ITDAT']).reshape(-1, 1))

In [ ]:
print("Scaler mean: ", sc_day.mean_)
print("Scaler scale: ", sc_day.scale_)

In [ ]:
sc_sorting = StandardScaler()
df['_SORTING'] = sc_sorting.fit_transform(np.array(df['_SORTING']).reshape(-1, 1))
df = df.sort_values(by = ['WERKS', 'EBELN', 'EBELP'])

df = df.loc[df['concept:name'] != 'Late Shipment'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.loc[df['concept:name'] != 'Late Delivery'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.loc[df['concept:name'] != 'Due Date passed'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.reset_index(drop= True)

n_act, onehot_encoder_act = one_hot_column(df['concept:name'])
n_wer, onehot_encoder_wer = one_hot_column(df['WERKS'])
n_nam, onehot_encoder_nam = one_hot_column(df['NAME1'])
n_mag, onehot_encoder_mag = one_hot_column(df['MATKL'])
n_mat, onehot_encoder_mat = one_hot_column(df['MATNR'])

# Time
A = [d.date() for d in df['time:timestamp']]
B = [d.date() for d in df['time:timestamp'].shift(-1)]
B[-1] = B[-2]
df['time:diff'] = np.busday_count(A, B)
df['time:diff'] = df['time:diff'].shift(1)
df.loc[0, 'time:diff'] = 0
idx = df.reset_index().groupby('case:concept:name')['index'].first().values
df.loc[df.index.isin(idx), 'time:diff'] = 0
sc = StandardScaler()
df['time:diff'] = sc.fit_transform(np.array(df['time:diff']).reshape(-1, 1))
df['concept:name'] = df.groupby('case:concept:name')['concept:name'].transform(lambda x: ','.join(x))
df['time:diff'] = df['time:diff'].astype(str)
df['time:diff'] = df.groupby('case:concept:name')['time:diff'].transform(lambda x: ','.join(x))
df['_SORTING'] = df['_SORTING'].astype(str)
df['_SORTING'] = df.groupby('case:concept:name')['_SORTING'].transform(lambda x: ','.join(x))

df = df[['case:concept:name', 'concept:name', 'RKDST', 'WERKS', 'time:diff', 'NAME1', 'MATKL', 'MATNR', 'MAVPR', 'THWMS', 'LATES','LATED','LATEP', '_SORTING', 
         'time:diff_BUDAT' , 'time:diff_BPDAT', 'time:diff_LEWED', 'time:diff_SEDAT', 'time:diff_LCWED', 'time:diff_ITDAT']].drop_duplicates().reset_index(drop = True)

traces= []
diff_time = []
sorting = []
for q in range(len(df)):
    string = np.array(df.loc[q, 'concept:name'].split(','),dtype = str).reshape(-1,1)
    onehot_encoded = onehot_encoder_act.transform(string)
    traces.append(onehot_encoded)
    diff_time.append(np.array(df.loc[q, 'time:diff'].split(','),dtype = float))
    sorting.append(np.array(df.loc[q, '_SORTING'].split(','),dtype = float))
print('Traccie: ',len(traces))

rda = df['RKDST']
werks_bef = df['WERKS']
name1_bef = df['NAME1']
matkl_bef = df['MATKL']
matnr_bef = df['MATNR']
mavp = df['MAVPR']
twms = df['THWMS']
lates = df['LATES']
lated = df['LATED']
latep = df['LATEP']
budat = df['time:diff_BUDAT']
bpdat = df['time:diff_BPDAT']
lewed = df['time:diff_LEWED']
sedat = df['time:diff_SEDAT']
lcwed = df['time:diff_LCWED']
itdat = df['time:diff_ITDAT']

werks = onehot_encoder_wer.transform(np.array(werks_bef).reshape(-1, 1))
name1 = onehot_encoder_nam.transform(np.array(name1_bef).reshape(-1, 1))
matkl = onehot_encoder_mag.transform(np.array(matkl_bef).reshape(-1, 1))
matnr = onehot_encoder_mat.transform(np.array(matnr_bef).reshape(-1, 1))

## RDA
for l in range(len(traces)):
    to_app = np.array([[rda[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# WERKS
for l in range(len(traces)):
    to_app = np.array([[werks[l]] * len(traces[l])]).reshape(len(traces[l]), n_wer)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# NAME1
for l in range(len(traces)):
    to_app = np.array([[name1[l]] * len(traces[l])]).reshape(len(traces[l]), n_nam)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# MATKL
for l in range(len(traces)):
    to_app = np.array([[matkl[l]] * len(traces[l])]).reshape(len(traces[l]), n_mag)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# MATNR
for l in range(len(traces)):
    to_app = np.array([[matnr[l]] * len(traces[l])]).reshape(len(traces[l]), n_mat)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# TIME
for l in range(len(traces)):
    to_app = diff_time[l].reshape(len(traces[l]),1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# MAVP
for l in range(len(traces)):
    to_app = np.array([[mavp[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# TWMS
for l in range(len(traces)):
    to_app = np.array([[twms[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LATES
for l in range(len(traces)):
    to_app = np.array([[lates[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LATED
for l in range(len(traces)):
    to_app = np.array([[lated[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LATEP
for l in range(len(traces)):
    to_app = np.array([[latep[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# BUDAT
for l in range(len(traces)):
    to_app = np.array([[budat[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# BPDAT
for l in range(len(traces)):
    to_app = np.array([[bpdat[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LEWED
for l in range(len(traces)):
    to_app = np.array([[lewed[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# SEDAT
for l in range(len(traces)):
    to_app = np.array([[sedat[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LCWED
for l in range(len(traces)):
    to_app = np.array([[lcwed[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# ITDAT
for l in range(len(traces)):
    to_app = np.array([[itdat[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# # SORTING
# for l in range(len(traces)):
#     to_app = sorting[l].reshape(len(traces[l]),1)
#     traces[l] = np.append(traces[l], to_app, axis = 1)

Process instances creation

In [ ]:
X = []
y = []

n_min = 1
n_zero = traces[0].shape[1]

for q in range(len(traces)):
    sequence = traces[q]
    if sequence.shape[0] > n_min:
        for windows in range(sequence.shape[0] - n_min):
            X.append(sequence[0:windows + n_min, :])
            y.append(sequence[windows + n_min: windows + n_min+1, :])
print('Numbero di X: ', len(X))

max_trace = find_max_list(traces)
print(max_trace)

for i in range(len(X)):
    num = max_trace - len(X[i])
    for n in range(num):
        X[i] = np.append(X[i], [[0] * n_zero],  axis=0)

X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=30)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

print('Train shape: ', len(X_train))
print('Val shape: ', len(X_test))
print('Test shape: ', len(X_val))

X_rda_train = X_train[:,0,n_act].reshape(-1,1)
X_rda_test = X_test[:,0,n_act].reshape(-1,1)
X_rda_val = X_val[:,0,n_act].reshape(-1,1)

X_wer_train = X_train[:,0,n_act+1 : n_act+1+n_wer]
X_wer_test = X_test[:,0,n_act+1 : n_act+1+n_wer]
X_wer_val = X_val[:,0,n_act+1 : n_act+1+n_wer]

X_nam_train = X_train[:,0,n_act+1+n_wer : n_act+1+n_wer+n_nam]
X_nam_test = X_test[:,0,n_act+1+n_wer : n_act+1+n_wer+n_nam]
X_nam_val = X_val[:,0,n_act+1+n_wer : n_act+1+n_wer+n_nam]

X_mag_train = X_train[:,0,n_act+1+n_wer+n_nam : n_act+1+n_wer+n_nam+n_mag]
X_mag_test = X_test[:,0,n_act+1+n_wer+n_nam : n_act+1+n_wer+n_nam+n_mag]
X_mag_val = X_val[:,0,n_act+1+n_wer+n_nam : n_act+1+n_wer+n_nam+n_mag]

X_mat_train = X_train[:,0,n_act+1+n_wer+n_nam+n_mag : n_act+1+n_wer+n_nam+n_mag+n_mat]
X_mat_test = X_test[:,0,n_act+1+n_wer+n_nam+n_mag : n_act+1+n_wer+n_nam+n_mag+n_mat]
X_mat_val = X_val[:,0,n_act+1+n_wer+n_nam+n_mag : n_act+1+n_wer+n_nam+n_mag+n_mat]

X_time_train = X_train[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,max_trace,1)
X_time_test = X_test[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,max_trace,1)
X_time_val = X_val[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,max_trace,1)

X_budat_train = X_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+6].reshape(-1,1)
X_budat_test = X_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+6].reshape(-1,1)
X_budat_val = X_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+6].reshape(-1,1)

X_bpdat_train = X_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+7].reshape(-1,1)
X_bpdat_test = X_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+7].reshape(-1,1)
X_bpdat_val = X_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+7].reshape(-1,1)

X_lewed_train = X_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+8].reshape(-1,1)
X_lewed_test = X_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+8].reshape(-1,1)
X_lewed_val = X_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+8].reshape(-1,1)

X_sedat_train = X_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+9].reshape(-1,1)
X_sedat_test = X_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+9].reshape(-1,1)
X_sedat_val = X_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+9].reshape(-1,1)

X_lcwed_train = X_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+10].reshape(-1,1)
X_lcwed_test = X_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+10].reshape(-1,1)
X_lcwed_val = X_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+10].reshape(-1,1)

X_itdat_train = X_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+11].reshape(-1,1)
X_itdat_test = X_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+11].reshape(-1,1)
X_itdat_val = X_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+11].reshape(-1,1)

# Y


y_bpdat_train = y_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+7].reshape(-1,1)
y_bpdat_test = y_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+7].reshape(-1,1)
y_bpdat_val = y_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+7].reshape(-1,1)

y_time_train = y_train[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,1)
y_time_test = y_test[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,1)
y_time_val = y_val[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,1)

y_mav_train = y_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+1].reshape(-1,1)
y_mav_test = y_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+1].reshape(-1,1)
y_mav_val = y_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+1].reshape(-1,1)

y_twm_train = y_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+2].reshape(-1,1)
y_twm_test = y_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+2].reshape(-1,1)
y_twm_val = y_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+2].reshape(-1,1)

y_lates_train = y_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+3].reshape(-1,1)
y_lates_test = y_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+3].reshape(-1,1)
y_lates_val = y_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+3].reshape(-1,1)

y_lated_train = y_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+4].reshape(-1,1)
y_lated_test = y_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+4].reshape(-1,1)
y_lated_val = y_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+4].reshape(-1,1)

y_latep_train = y_train[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+5].reshape(-1,1)
y_latep_test = y_test[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+5].reshape(-1,1)
y_latep_val = y_val[:,0, n_act+1+n_wer+n_nam+n_mag+n_mat+5].reshape(-1,1)

X_train = X_train[:,:,:n_act]
X_test = X_test[:,:,:n_act]
X_val = X_val[:,:,:n_act]
# Commentare riga per più output
y_train = y_train[:,:,:n_act].reshape(-1,n_act)
y_test = y_test[:,:,:n_act].reshape(-1,n_act)
y_val = y_val[:,:,:n_act].reshape(-1,n_act)

class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(y_train.argmax(axis = 1)), y= y_train.reshape(-1,n_act).argmax(axis = 1))
#class_weights = np.insert(class_weights, 3, 0)
np.save('class_weights.npy', class_weights)
loss = weighted_categorical_crossentropy(class_weights)

# from sklearn.utils.class_weight import compute_class_weight
# class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(X_train.reshape(-1,11)[np.any(X_train.reshape(-1,11) != [0]*11, axis=1)].argmax(axis = 1)), y= X_train.reshape(-1,11)[np.any(X_train.reshape(-1,11) != [0]*11, axis=1)].argmax(axis = 1))
# #class_weights = np.insert(class_weights, 3, 0)

# convertire to integer encoding
X_wer_train_lab = X_wer_train.argmax(axis = 1).reshape(-1,1)
X_wer_val_lab = X_wer_val.argmax(axis = 1).reshape(-1,1)
X_wer_test_lab = X_wer_test.argmax(axis = 1).reshape(-1,1)

X_nam_train_lab = X_nam_train.argmax(axis = 1).reshape(-1,1)
X_nam_val_lab = X_nam_val.argmax(axis = 1).reshape(-1,1)
X_nam_test_lab = X_nam_test.argmax(axis = 1).reshape(-1,1)

X_mag_train_lab = X_mag_train.argmax(axis = 1).reshape(-1,1)
X_mag_val_lab = X_mag_val.argmax(axis = 1).reshape(-1,1)
X_mag_test_lab = X_mag_test.argmax(axis = 1).reshape(-1,1)

X_mat_train_lab = X_mat_train.argmax(axis = 1).reshape(-1,1)
X_mat_val_lab = X_mat_val.argmax(axis = 1).reshape(-1,1)
X_mat_test_lab = X_mat_test.argmax(axis = 1).reshape(-1,1)


# Models

In [ ]:
initializer = tf.keras.initializers.GlorotUniform(seed = 1)

In [ ]:
name = 'model_suffix_act_2'

main_input = Input(shape=(max_trace, n_act), name='main_input')
m1 = Masking(mask_value=0)(main_input)
l1 = SimpleRNN(32, return_sequences=False, dropout = 0, kernel_initializer=initializer)(m1)
time_input = Input(shape=(max_trace, 1), name='time_input')
m2 = Masking(mask_value=0)(time_input)
l2 = SimpleRNN(16, return_sequences=False, dropout = 0, kernel_initializer=initializer)(m2)

c1 = Concatenate()([l1, l2])
d1 = Dense(16, activation = 'relu', kernel_initializer=initializer)(c1)

pr_input = Input(shape=(1), name='pr_input')
we_input = Input(shape=(n_wer), name = 'we_input')
na_input = Input(shape=(n_nam), name = 'na_input')
mg_input = Input(shape=(n_mag), name='mg_input')
mt_input = Input(shape=(n_mat), name = 'mt_input')

c2 = Concatenate()([pr_input,
                    we_input,
                    na_input,
                    mg_input,
                    mt_input,                    
                    d1])

dp1 = Dropout(0.2)(c2)
d_act = Dense(n_act, kernel_initializer=initializer)(dp1)
act_output = Softmax(name='act_output')(d_act)

d_mav = Dense(1, kernel_initializer=initializer)(c2)
mav_output = keras.layers.Activation(activations.sigmoid, name='mav_output')(d_mav)

model = Model([main_input,
               time_input,
               pr_input,
               we_input,
               na_input,
               mg_input,
               mt_input
               ],
               [act_output,
                mav_output,
                ])
opt = keras.optimizers.Adam(learning_rate=0.0001)
loss = weighted_categorical_crossentropy(class_weights)
model.compile(loss={'act_output': loss, #'time_output': 'mean_absolute_error',
                    'mav_output': 'binary_crossentropy',
                    },
             optimizer=opt)
early_stopping = EarlyStopping(monitor='val_act_output_loss', patience=10)
mcp_save = ModelCheckpoint("model/" + name + ".h5", save_best_only=True, monitor='val_loss', mode='min')
model.summary()

In [ ]:
from livelossplot import PlotLossesKeras

In [ ]:
history = model.fit([X_train,
                     X_time_train,
                     X_rda_train,
                     X_wer_train,
                     X_nam_train,
                     X_mag_train,
                     X_mat_train
                     ],
                     [y_train,
                      y_mav_train],
                    epochs=100,
                    batch_size=64,
                    shuffle=True,
                    validation_data=([X_val,
                                      X_time_val,
                                      X_rda_val,
                                      X_wer_val,
                                      X_nam_val,
                                      X_mag_val,
                                      X_mat_val
                     ],
                     [y_val, y_mav_val
                      ]),
                    callbacks = [early_stopping, mcp_save, PlotLossesKeras()])


In [ ]:
model = keras.models.load_model('model/model_suffix_act.h5', custom_objects = {'loss' : loss})
model.summary()

In [ ]:
yhat, yhat_mavp = model.predict([X_test,
                                 X_time_test,
                                 X_rda_test,
                                 X_wer_test,
                                 X_nam_test,
                                 X_mag_test,
                                 X_mat_test
                                    ])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
T5_labels = onehot_encoder_act.inverse_transform(np.identity(n_act))[:,0]

cm = confusion_matrix(y_test.argmax(axis = 1), yhat.argmax(axis = 1), labels = np.identity(n_act).argmax(axis = 1), normalize = 'true')
df_cm = pd.DataFrame(cm, range(n_act), range(n_act))
sns.set(font_scale = 1.4)
g = sns.heatmap(df_cm, annot = True, annot_kws={'size' : 10},
                xticklabels = T5_labels, yticklabels= T5_labels,
                #vmin = 0, vmax = 1, fmt=".0%",
                cmap ='Blues',
                square = True, linewidths=1.4, cbar_kws={"shrink" : .8}, ax = ax)
g.set_xticklabels(g.get_xmajorticklabels(), fontsize = 10)
g.set_yticklabels(g.get_xmajorticklabels(), fontsize = 10) 
plt.xticks(rotation=90)
plt.show()

In [ ]:
def X_2_activity(X, y):
    X_2 = X.copy()
    for i in range(len(X_2)):
        for q in range(max_trace):
            if (X_2[i][q] == [0] * n_act).all():
                X_2[i][q] = y[i]
                break
    return X_2

In [ ]:
name = 'model_suffix_time'

main_input = Input(shape=(max_trace, n_act), name='main_input')
m1 = Masking(mask_value=0)(main_input)
l1 = SimpleRNN(32, return_sequences=False, dropout = 0, kernel_initializer=initializer)(m1)
time_input = Input(shape=(max_trace, 1), name='time_input')
m2 = Masking(mask_value=0)(time_input)
l2 = SimpleRNN(16, return_sequences=False, dropout = 0, kernel_initializer=initializer)(m2)

c1 = Concatenate()([l1, l2])
d1 = Dense(16, activation = 'relu', kernel_initializer=initializer)(c1)

pr_input = Input(shape=(1), name='pr_input')
mg_input = Input(shape=(n_mag), name='mg_input')

c2 = Concatenate()([pr_input, mg_input, d1])

# dp1 = Dropout(0.2)(c2)
# d_act = Dense(n_act, kernel_initializer=initializer)(dp1)
# act_output = Softmax(name='act_output')(d_act)

time_output = Dense(1, name='time_output', kernel_initializer=initializer)(c2)

# d_mav = Dense(1, kernel_initializer=initializer)(c2)
# mav_output = keras.layers.Activation(activations.sigmoid, name='mav_output')(d_mav)

# d_twm = Dense(1, kernel_initializer=initializer)(c2)
# twm_output = keras.layers.Activation(activations.sigmoid, name='twm_output')(d_twm)


model = Model([main_input, time_input,
               pr_input, mg_input,
               ],
               [time_output
                # twm_output, latep_output
                ])#, lates_output, lated_output, latep_output])
opt = keras.optimizers.Adam(learning_rate=0.0001)
loss = weighted_categorical_crossentropy(class_weights)
model.compile(loss={
    #'act_output': loss, 
    'time_output': 'mean_absolute_error',
    #                'mav_output': 'binary_crossentropy', #'twm_output': 'binary_crossentropy',
                    },
             optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
mcp_save = ModelCheckpoint("model/" + name + ".h5", save_best_only=True, monitor='val_loss', mode='min')
model.summary()

In [ ]:
from livelossplot import PlotLossesKeras

In [ ]:
X_2_train = X_2_activity(X_train, y_train)
X_2_val = X_2_activity(X_val, y_val)

In [ ]:
history = model.fit([X_2_train, X_time_train,
                     X_rda_train, X_mag_train,
                     ],
                     [y_time_train# , y_twm_train, y_latep_train
                      ],# , y_lates_train, y_lated_train, y_bpdat_train],
                    epochs=100,
                    batch_size=64,
                    shuffle=True,
                    validation_data=([X_2_val, X_time_val,
                     X_rda_val,X_mag_val
                     ],
                     [y_time_val# , y_twm_val, y_latep_val
                      ]),# , y_lates_val, y_lated_val, y_bpdat_val]),
                    callbacks = [early_stopping, mcp_save, PlotLossesKeras()])
